In [ ]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!pip install cupy-cuda117  # The version depends on the CUDA version available in Colab.
!pip install fastapi python-multipart uvicorn
!pip install -U kaleido
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.7/81.7 MB 12.5 MB/s eta 0:00:00


In [44]:
import numpy as np
import plotly.graph_objects as go
import imageio
from tqdm.notebook import tqdm
from scipy.interpolate import RegularGridInterpolator

# Constants
hbar = 1
m = 1
N = 50  # Reduced for 3D computation
dt = 0.1
V0 = 1000

# 3D Grids
x = np.linspace(-10, 10, N)
y = np.linspace(-10, 10, N)
z = np.linspace(-10, 10, N)
X, Y, Z = np.meshgrid(x, y, z)

# 3D Wavefunction (Gaussian wave packet)
sigma = 1.0
k0 = 2.0
psi_x = np.exp(-(X**2 + Y**2 + Z**2) / (2 * sigma**2)) * np.exp(1j * k0 * X)
psi_x /= np.linalg.norm(psi_x)

# Time evolution operator in momentum space
KX, KY, KZ = np.meshgrid(np.fft.fftfreq(N, d=x[1] - x[0]) * 2 * np.pi,
                         np.fft.fftfreq(N, d=y[1] - y[0]) * 2 * np.pi,
                         np.fft.fftfreq(N, d=z[1] - z[0]) * 2 * np.pi)

U_k = np.exp(-1j * (KX**2 + KY**2 + KZ**2) * dt / (2 * m))

# Time evolution operator in position space (excluding the potential)
R = np.sqrt(X**2 + Y**2 + Z**2)
V = np.where(R < 1, V0, 0)
U_x = np.exp(-1j * V * dt / (2 * hbar))

# Function for time step evolution
def time_step(psi_x):
    psi_k = np.fft.fftn(psi_x)
    psi_k *= U_k
    psi_x = np.fft.ifftn(psi_k)
    psi_x *= U_x
    return psi_x

# Parameters for the animation
number_of_time_steps = 18
frames = []

# Generate and save isosurface plots for each time step
for t in tqdm(range(number_of_time_steps), desc="Generating frames"):
    psi_x = time_step(psi_x)
    abs_psi_x = np.abs(psi_x)**2  # Probability density

    # Normalize and amplify the probability density
    max_val = np.max(abs_psi_x)
    scaled_abs_psi_x = 100000 * (abs_psi_x / max_val)  # Amplify by a factor of 100000

    # Cap the scaled values to avoid extreme highs
    cap_value = 5.0  # Adjust this cap value as needed
    scaled_abs_psi_x = np.minimum(scaled_abs_psi_x, cap_value)

    # Set isomin and isomax values
    isomin = 0.0001  # Lower bound for isosurface
    isomax = 0.5     # Upper bound for isosurface

    custom_colorscale = [
      [0.0, 'black'],      # Set the lowest values to black
      [0.1, 'darkblue'],   # Gradually transition to dark blue
      [0.2, 'blue'],       # Transition to blue
      [0.3, 'lightblue'],  # Transition to light blue
      [0.4, 'lightgreen'], # Transition to light green
      [0.5, 'green'],      # Transition to green
      [0.6, 'yellowgreen'],# Transition to yellow-green
      [0.7, 'yellow'],     # Transition to yellow
      [0.8, 'orange'],     # Transition to orange
      [0.9, 'darkorange'], # Transition to dark orange
      [1.0, 'red']         # Highest values are represented in red
    ]

    # Create an isosurface plot with custom color scale and adjusted opacity
    fig = go.Figure(data=go.Isosurface(
        x=X.flatten(),
        y=Y.flatten(),
        z=Z.flatten(),
        value=scaled_abs_psi_x.flatten(),
        isomin=isomin,
        isomax=isomax,
        opacity=0.3,
        surface_count=15,
        colorscale=custom_colorscale
    ))

    # Add a 2D heatmap as a colorbar
    scale_bar = go.Heatmap(
        z=[[isomin, isomax]],
        xaxis='x2',
        yaxis='y2',
        colorscale='RdBu',
        showscale=False
    )
    fig.add_trace(scale_bar)

    # Adjust the camera settings for a zoomed-in view
    camera_scale = 1.5
    camera = dict(
        eye=dict(x=camera_scale, y=camera_scale, z=camera_scale),  # Closer camera position
        center=dict(x=0, y=0, z=0),      # Center of the plot
        up=dict(x=0, y=0, z=1)           # Orientation of the up vector
    )

    # Update layout
    fig.update_layout(
        scene_camera=camera,
        title=f"Quantum Wave Packet - Time Step: {t}",
        margin=dict(l=0, r=0, b=0, t=30),
        scene=dict(
            xaxis=dict(showbackground=False, showticklabels=False, visible=False, showline=False),
            yaxis=dict(showbackground=False, showticklabels=False, visible=False, showline=False),
            zaxis=dict(showbackground=False, showticklabels=False, visible=False, showline=False),
            bgcolor='black'
        ),
        xaxis2=dict(
            domain=[0.85, 0.9],
            showticklabels=False
        ),
        yaxis2=dict(
            domain=[0.1, 0.9],
            showticklabels=False
        ),
        annotations=[
            dict(
                x=0.85,
                y=0.95,
                xref='paper',
                yref='paper',
                text='Max Density',
                showarrow=False
            ),
            dict(
                x=0.85,
                y=0.05,
                xref='paper',
                yref='paper',
                text='Min Density',
                showarrow=False
            )
        ]
    )



    # Save the plot as an image
    frame_filename = f'frame_{t}.png'
    frames.append(frame_filename)
    fig.write_image(frame_filename)


# Create an MP4 video from the saved images
with imageio.get_writer('quantum_wave_packet.mp4', fps=20) as writer:
    for frame in tqdm(frames, desc="Creating MP4"):
        image = imageio.imread(frame)
        writer.append_data(image)

# Cleanup: remove the individual frame files
import os
for frame in frames:
    os.remove(frame)


Generating frames:   0%|          | 0/18 [00:00<?, ?it/s]

Creating MP4:   0%|          | 0/18 [00:00<?, ?it/s]

<ipython-input-44-942e4fb6375a>:162: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.

